<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/swiss_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers rank_bm25

In [3]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch

In [4]:
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

In [ ]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('swiss_judgment_prediction', 'de')

In [6]:
passages = dataset['train']['text']

In [ ]:
passages

In [8]:
print("passages:", len(passages))

passages: 35458


In [ ]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') # load bi-encoder
bi_encoder.max_seq_length = 256     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
# We encode all passages into our vector space. ~5 minutes
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

In [12]:
# We also compare the results to lexical search (keyword search). Here, we use 
# the BM25 algorithm which is implemented in the rank_bm25 package.
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

In [13]:
# We lower case our text and remove stop-words from indexing
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for passage in tqdm(passages):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)


  0%|          | 0/35458 [00:00<?, ?it/s]

In [19]:
# This function will search all wikipedia articles for passages that
# answer the query
def search(query):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -5)[-5:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    print("Top-5 lexical search (BM25) hits")
    for hit in bm25_hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, passages[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    print("\n-------------------------\n")
    print("Top-5 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    print("\n-------------------------\n")
    print("Top-5 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " ")))

In [20]:
search(query = " Krankentaggelder, wobei die Leistungen bis am 30. September 2012 auf Grundlage einer Arbeitsunf\u00e4higkeit von 100% und danach basierend auf einer Arbeitsunf\u00e4higkeit von 55% erbracht wurden.")

Input question:  Krankentaggelder, wobei die Leistungen bis am 30. September 2012 auf Grundlage einer Arbeitsunfähigkeit von 100% und danach basierend auf einer Arbeitsunfähigkeit von 55% erbracht wurden.
Top-5 lexical search (BM25) hits
	66.288	Sachverhalt: Sachverhalt: A. Die 1955 geborene C._ verletzte sich am 3. Juni 1997 bei der Arbeit als Hausangestellte in der Klinik X._ in Y._ am rechten Arm. Die Abklärungen auf der Handchirurgischen Abteilung des Spitals Z._ führten zur Diagnose einer traumatisierten Daumen-Sattelgelenksarthrose rechts (Bericht vom 25. Februar 1998). Die Zürich Versicherungs-Gesellschaft (nachfolgend: Zürich), bei welcher C._ obligatorisch unfallversichert war, kam für die Heilungskosten auf und richtete bis 30. Mai 2002 Taggelder auf der Grundlage einer Arbeitsunfähigkeit von 100 % aus. Wegen Verlusts der Akten zog die Zürich zur Beurteilung ihrer Leistungspflicht die IV-Akten bei (interner Bericht vom 16. März 2004). Mit Verfügung vom 26. März 2004 stellte s